#### Setup the notebook

In [1]:
#************************************************************************************************
# Imports
#************************************************************************************************
import os
from dotenv import load_dotenv
import pandas as pd
from openai import AzureOpenAI
import json
import time

# Import the assistant tools
from tools import get_list_of_dockets

#************************************************************************************************
# Load environment variables from a .env file and set local variables
#************************************************************************************************
load_dotenv()

AZURE_OPENAI_API_ENDPOINT = os.getenv("AZURE_OPENAI_API_ENDPOINT")
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")
RA_ASSISTANT_ID = "asst_JLRYLt14l5wWRbpC6Y1VXwbc"


#************************************************************************************************
# Create client instance of the AzureOpenAI class
#************************************************************************************************
AOAI = AzureOpenAI(
            azure_endpoint=AZURE_OPENAI_API_ENDPOINT,
            api_key=AZURE_OPENAI_API_KEY,
            api_version=AZURE_OPENAI_API_VERSION
        )

#************************************************************************************************
# Setup utility functions
#************************************************************************************************

# Pretty printing JSON
def show_json(obj):
    display(json.loads(obj.model_dump_json()))

# Pretty printing messages
def pretty_print(messages):
    print("# Messages")
    for m in messages:
        print(f"{m.role}: {m.content[0].text.value}")
    print()

def wait_on_run(run, thread):
    while run.status == "queued" or run.status == "in_progress":
        run = AOAI.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)  
        time.sleep(0.5)
    return run

def submit_message(assistant_id, thread, user_message):
    AOAI.beta.threads.messages.create(thread_id=thread.id, role="user", content=user_message)
    return AOAI.beta.threads.runs.create(thread_id=thread.id, assistant_id=assistant_id)

def get_response(thread):
    return AOAI.beta.threads.messages.list(thread_id=thread.id, order="asc")

def create_thread_and_run(user_input):
    thread = AOAI.beta.threads.create()
    run = submit_message(RA_ASSISTANT_ID, thread, user_input)
    return thread, run



Making API Call to: https://api.regulations.gov/v4/documents/USPS-2016-0169-0001
With parameters: {'api_key': '4p2Hpwlq1SJ5kOhayfhqeI0D1RtDOo70d8azejwL'}
API Call Successful!
<html>
<head>
<title>Federal Register, Volume 81 Issue 172 (Tuesday, September 6, 2016)</title>
</head>
<body><pre>
[Federal Register Volume 81, Number 172 (Tuesday, September 6, 2016)]
[Notices]
[Page 61259]
From the Federal Register Online via the Government Publishing Office [<a href="http://www.gpo.gov">www.gpo.gov</a>]
[FR Doc No: 2016-21267]


-----------------------------------------------------------------------

POSTAL SERVICE


Product Change--Priority Mail Express and Priority Mail 
Negotiated Service Agreement

AGENCY: Postal Service<SUP>TM</SUP>.

ACTION: Notice.

-----------------------------------------------------------------------

SUMMARY: The Postal Service gives notice of filing a request with the 
Postal Regulatory Commission to add a domestic shipping services 
contract to the list of Negotia

In [2]:
thread, run = create_thread_and_run("Are there any recent proposed regulations from the State Department dealing with ITAR.")
run = wait_on_run(run, thread)
run.status

'requires_action'

In [ ]:
show_json(run)

In [3]:
# Extract single tool call
tool_call = run.required_action.submit_tool_outputs.tool_calls[0]
name = tool_call.function.name
arguments = json.loads(tool_call.function.arguments)

print("Function Name:", name)
print("Function Arguments:")
arguments

Function Name: get_list_of_dockets
Function Arguments:


{'agencyId': 'DOS', 'searchTerm': 'ITAR', 'afterPostedDate': '2023-01-01'}

In [6]:
function_to_call = globals()[name]
result = function_to_call(**arguments)

print("Function Output:")
print(result)

Making API Call to: https://api.regulations.gov/v4/documents
With parameters: {'api_key': '4p2Hpwlq1SJ5kOhayfhqeI0D1RtDOo70d8azejwL', 'filter[documentType]': 'Proposed Rule', 'sort': '-postedDate', 'filter[agencyId]': 'DOS', 'filter[searchTerm]': 'ITAR', 'filter[postedDate][ge]': '2023-01-01'}
API Call Successful!
Function Output:
{'data': [{'id': 'DOS-2024-0013-0001', 'type': 'documents', 'attributes': {'documentType': 'Proposed Rule', 'lastModifiedDate': '2024-05-01T13:18:56Z', 'highlightedContent': '', 'frDocNum': '2024-08829', 'withdrawn': False, 'agencyId': 'DOS', 'commentEndDate': '2024-06-01T03:59:59Z', 'title': 'International Traffic in Arms Regulations: Exemption for Defense Trade and Cooperation Among Australia, the United Kingdom, and the United States', 'postedDate': '2024-05-01T04:00:00Z', 'docketId': 'DOS-2024-0013', 'subtype': None, 'commentStartDate': '2024-05-01T04:00:00Z', 'openForComment': True, 'objectId': '090000648652cd63'}, 'links': {'self': 'https://api.regulati

In [ ]:
run = AOAI.beta.threads.runs.submit_tool_outputs(
    thread_id=thread.id,
    run_id=run.id,
    tool_outputs=[
        {
            "tool_call_id": tool_call.id,
            "output": json.dumps(result),
        }
    ],
)
show_json(run)

In [ ]:
run = wait_on_run(run, thread)
run.status

In [ ]:
# Extract single tool call
tool_call = run.required_action.submit_tool_outputs.tool_calls[0]
name = tool_call.function.name
arguments = json.loads(tool_call.function.arguments)

print("Function Name:", name)
print("Function Arguments:")
arguments

In [ ]:
responses = get_data_from_last_10K_filing_by_section(ticker="MSFT",section_path="Role_DisclosureEARNINGSPERSHARETables")
print("Responses:", responses)

In [ ]:
run = AOAI.beta.threads.runs.submit_tool_outputs(
    thread_id=thread.id,
    run_id=run.id,
    tool_outputs=[
        {
            "tool_call_id": tool_call.id,
            "output": json.dumps(responses),
        }
    ],
)
show_json(run)

In [ ]:
run = wait_on_run(run, thread)
pretty_print(get_response(thread))